<a href="https://colab.research.google.com/github/liuyx211/CS230-Project/blob/main/template_hyper_yolov5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import matplotlib.pyplot as plt
import re
from IPython.display import display

%matplotlib inline

random.seed(108)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dir_prefix = "/content/drive/MyDrive/newCS230/yolo_data"

In [ ]:
# image_path = "/content/drive/MyDrive/newCS230/newdataset"
# yolo_path ="/content/drive/MyDrive/newCS230/new_yolo_labels"

In [ ]:
# # Read images and annotations
# images = [os.path.join(image_path, x.replace("txt", "jpeg")) for x in os.listdir(yolo_path) if x[-3:] == "txt"]
# annotations = [os.path.join(yolo_path, x) for x in os.listdir(yolo_path) if x[-3:] == "txt"]

# images.sort()
# annotations.sort()

# # Split the dataset into train-valid-test splits 
# train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)
# val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

In [ ]:
# !rm -r $dir_prefix
# !mkdir $dir_prefix
# %cd $dir_prefix

# !mkdir images annotations
# !mkdir images/train images/val images/test annotations/train annotations/val annotations/test

# #Utility function to copy images 
# def copy_files_to_folder(list_of_files, destination_folder):
#     for f in list_of_files:
#         try:
#             shutil.copy(f, destination_folder)
#         except:
#             print(f)
#             assert False

# # Move the splits into their folders
# copy_files_to_folder(train_images, 'images/train')
# copy_files_to_folder(val_images, 'images/val/')
# copy_files_to_folder(test_images, 'images/test/')
# copy_files_to_folder(train_annotations, 'annotations/train/')
# copy_files_to_folder(val_annotations, 'annotations/val/')
# copy_files_to_folder(test_annotations, 'annotations/test/')
# mv annotations labels

In [ ]:
%cd /content
!pwd

/content
/content


In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 10041, done.
remote: Total 10041 (delta 0), reused 0 (delta 0), pack-reused 10041
Receiving objects: 100% (10041/10041), 10.35 MiB | 21.62 MiB/s, done.
Resolving deltas: 100% (6968/6968), done.
/content/yolov5
     |████████████████████████████████| 596 kB 13.1 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
%cd data/
!pwd

/content/yolov5/data
/content/yolov5/data


In [ ]:
!touch vac.yaml

In [ ]:
with open('vac.yaml', 'w') as writefile:
    writefile.write("train: " + dir_prefix + '/images/train' + '\n')
    writefile.write("val: " + dir_prefix + '/images/val' + '\n')
    writefile.write("test: " + dir_prefix + '/images/test' + '\n')
    writefile.write('nc: 4\n')
    writefile.write("names: [\"card\",\"header\", \"logo\",\"history\"]")

In [ ]:
%cd hyps/
!pwd

/content/yolov5/data/hyps
/content/yolov5/data/hyps


In [ ]:
!cp hyp.scratch.yaml hyp.vac.yaml

In [ ]:
with open('hyp.vac.yaml', 'r+') as f:
    data = f.read()
    f.seek(0)
    f.write(re.sub(r'lr0: 0.01', r'lr0: 0.01', data))
    f.truncate()


In [ ]:
 with open('hyp.vac.yaml', 'r+') as f:
    data = f.read()
    f.seek(0)   
    f.write(re.sub(r'momentum: 0.937', r'momentum: 0.937', data))
    f.truncate()

In [ ]:
%cd ../..

/content/yolov5


In [ ]:
!python train.py --img 640 --cfg yolov5m.yaml --hyp hyp.vac.yaml --batch 32 --epochs 100 --data vac.yaml --weights yolov5m.pt --name yolo_vac_det

train: weights=yolov5s.pt, cfg=yolov5s.yaml, data=vac.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=100, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=yolo_vac_det, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-94-g47fac9f torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.

In [ ]:
!python detect.py --weights runs/train/yolo_vac_det/weights/best.pt --img 640 --conf 0.5 --source /content/drive/MyDrive/newCS230/yolo_data/images/test

detect: weights=['runs/train/yolo_vac_det/weights/best.pt'], source=/content/drive/MyDrive/newCS230/yolo_data/images/test, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-94-g47fac9f torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model Summary: 213 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
image 1/18 /content/drive/.shortcut-targets-by-id/1dPAqmfuNnHWqCr0aFttoNvezdjphyJg7/newCS230/yolo_data/images/test/103.jpeg: 480x640 1 card, 1 header, 1 logo, 1 history, Done. (0.032s)
image 2/18 /content/drive/.shortcut-targets-by-id/1dPAqmfuNnHWqCr0aFttoNvezdjphyJg7/newCS230/yolo_data/images/test/109.jpeg: 544x640 1 card, 1 header, 1 logo, 1 history, D

In [ ]:
!cp -r /content/yolov5/runs /content/drive/MyDrive/newCS230/Nov16_correct_label_run/